In [49]:
import numpy as np
import math
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import fsolve


In [43]:
def get_Rs_Rsh(Voc,Isc,Vmp,Imp,Pmp,Efficiency,T,Area):
    vp = Vmp/Voc
    ip = Imp/Isc

    FF = vp*ip
    # print('FF = ',FF)

    # Define the function to find roots
    def equation_m(x):
        return x*math.log10(vp) + math.log10(x+1)

    # Use fsolve to find the root
    initial_guess_m = 10
    solution_m = fsolve(equation_m,initial_guess_m)

    m = solution_m[0]
    # print('m = ',m)

    # Define the function to find roots
    def equation_gamma(x):
        return vp*(1-((1-x)*vp)-x*(vp**m)) - FF

    initial_guess_gamma = 1
    solution_gamma = fsolve(equation_gamma,initial_guess_gamma)

    gamma = solution_gamma[0]
    # print('gamma = ',gamma)

    v_check = np.linspace(0,1,500)
    i_check0 = 1 - ((1-gamma)*v_check) - gamma*(v_check**m)

    # plt.figure(1)
    # plt.plot(v_check,i_check0)


    # Define the system of equations
    def equations(vars):
        x, y = vars
        eq1 = (x+1)**(-1/x) - 0.05*(1-y) - vp
        eq2 = vp*(1 - ((1-y)*vp) - y*(vp**x)) - FF
        return [eq1, eq2]

    # Initial guess for (x, y)
    initial_guess = (10, 1)

    # Use fsolve to find the roots
    solution_m_gamma = fsolve(equations, initial_guess)

    m_actual = solution_m_gamma[0]
    gamma_actual = solution_m_gamma[1]

    # print('m_actual = ',m_actual,'gamma_actual = ',gamma_actual)

    v_check = np.linspace(0,1,500)
    i_check1 = 1 - ((1-gamma_actual)*v_check) - gamma_actual*(v_check**m_actual)

    # plt.figure(2)
    # plt.title('I-V Characteristic obtain from the given parameters')
    # plt.xlabel('V/Voc')
    # plt.ylabel('I/Isc')
    # plt.plot(v_check,i_check1)

    Rsh = Voc/(Isc*(1-gamma_actual))
    Rsh_total = Rsh*Area

    # Assumption zeta = 1, theta = 0.6 and Temp = 25 degC
    zeta = 1
    VT = (1.38*10**-23)*T/(1.6*10**-19)
    K = 1/(zeta*VT)
    theta = 0.6

    Rs = ((Voc*K/m)-1) / (theta*gamma*Isc*K)
    Rs_total = Rs*Area

    Jsc = Isc/Area

    Jph = Jsc*(1 + (Rs_total/Rsh_total))

    J0 = 0.1*Jph

    N = 9
    Rx = 2*Rs_total*N


    # def equation_J0(x):
    #     return math.log(Jph - (1/(K*Rsh_total))*math.log(Jph/x)) - K*x*Voc

    # # Use fsolve to find the root
    # initial_guess_J0 = 0.0001
    # solution_J0 = fsolve(equation_m,initial_guess_J0)

    # J0 = solution_J0[0]


    # print('Rshunt for the complete array = ',Rsh_total,' ohms')
    # print('Rseries for the complete array = ',Rs,' ohms')
    # print('\n')

    return Rs_total,Rsh_total,gamma_actual,m_actual,FF,Jph,J0,Rx

In [44]:
Voc = [0.680,0.679,0.678,0.677,0.676,0.675,0.674,0.673,0.672,0.671]
Isc = [11.253,11.247,11.233,11.208,11.185,11.172,11.153,11.131,11.115,11.104]
Vmp = [0.594,0.593,0.592,0.591,0.590,0.589,0.587,0.586,0.585,0.584]
Imp = [10.739,10.721,10.700,10.682,10.658,10.628,10.604,10.589,10.553,10.496]
Pmp = [6.28,6.25,6.22,6.20,6.17,6.14,6.11,6.09,6.06,6.03]
Efficiency = [22.9,22.8,22.7,22.6,22.5,22.4,22.3,22.2,22.1,22.0]
T = 25 + 273
Area = (166/10)**2

rs = np.zeros(len(Voc))
rsh = np.zeros(len(Voc))
Gamma = np.zeros(len(Voc))
M = np.zeros(len(Voc))
ff = np.zeros(len(Voc))
jph = np.zeros(len(Voc))
j0 = np.zeros(len(Voc))
rx = np.zeros(len(Voc))


for i in range(len(Voc)):
    result = get_Rs_Rsh(Voc[i],Isc[i],Vmp[i],Imp[i],Pmp[i],Efficiency[i],T,Area)
    rs[i] = result[0]
    rsh[i] = result[1]
    Gamma[i] = result[2]
    M[i] = result[3]
    ff[i] = result[4]
    jph[i] = result[5]
    j0[i] = result[6]
    rx[i] = result[7]

    
    print('Rshunt for the complete array = ',rsh[i],' ohms')
    print('Rseries for the complete array = ',rs[i],' ohms')
    print('\n')

Rshunt for the complete array =  2405.6894528807966  ohms
Rseries for the complete array =  0.12160366060997184  ohms


Rshunt for the complete array =  2015.960842666562  ohms
Rseries for the complete array =  0.12273334385842637  ohms


Rshunt for the complete array =  1839.7083390978812  ohms
Rseries for the complete array =  0.12389658893388081  ohms


Rshunt for the complete array =  2019.607278815767  ohms
Rseries for the complete array =  0.12500810784285146  ohms


Rshunt for the complete array =  1988.8171854039938  ohms
Rseries for the complete array =  0.12621138946971477  ohms


Rshunt for the complete array =  1609.0923399688706  ohms
Rseries for the complete array =  0.1275237230335  ohms


Rshunt for the complete array =  1658.8194953504085  ohms
Rseries for the complete array =  0.15004808370017533  ohms


Rshunt for the complete array =  1807.936410285003  ohms
Rseries for the complete array =  0.1512017643394803  ohms


Rshunt for the complete array =  1438.3664099154

In [45]:
print(Vmp[1],' ',Imp[1],' ',Pmp[1],' ',Efficiency[1],' ',rs[1],' ',rsh[1],' ',Gamma[1],' ',M[1],' ',ff[1],' ',jph[1],' ',j0[1],' ',rx[1])

0.593   10.721   6.25   22.8   0.12273334385842637   2015.960842666562   0.9917478490618599   23.788682142353384   0.83249861556929   0.04081755235358106   0.0040817552353581065   2.2092001894516744


In [46]:
print('j0 = ',j0)
print('jph = ',jph)
print('rx = ',rx)

j0 =  [0.00408389 0.00408176 0.0040767  0.00406761 0.00405926 0.00405461
 0.00404776 0.00403975 0.00403403 0.00403025]
jph =  [0.04083891 0.04081755 0.04076701 0.04067606 0.04059265 0.04054611
 0.04047761 0.04039748 0.04034033 0.0403025 ]
rx =  [2.18886589 2.20920019 2.2301386  2.25014594 2.27180501 2.29542701
 2.70086551 2.72163176 2.74905875 2.78338529]
